In [1]:
from __future__ import print_function

import nltk
import keras
import pandas as pd
import numpy as np

from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, Conv2D, GlobalMaxPooling2D
from keras.datasets import imdb
from keras.datasets import mnist
from keras.optimizers import RMSprop
from keras.datasets import reuters

Using Theano backend.


In [22]:
raw_data = pd.read_csv('C:\Users\pedro.castanha\Downloads\ML_Gabinete_Digital.csv',
                       error_bad_lines=False,
                       sep='\t',
                       encoding='utf_8')

stop = set(stopwords.words('portuguese'))

x_vectorized = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words=stop)
x_vectorized.fit(raw_data.Text)
x_train = x_vectorized.transform(raw_data.Text)
y_train = raw_data.Class

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [24]:
xx = sequence.pad_sequences(x_train.toarray(), maxlen=maxlen)
xx_test = sequence.pad_sequences(x_test.toarray(), maxlen=maxlen)

xx.shape

(504L, 400L)

In [ ]:
# set parameters:
max_features = 10000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

print('Loading data...')
#(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
#print(len(x_train), 'train sequences')
#print(len(x_test), 'test sequences')

#print('Pad sequences (samples x time)')
#x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
#x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
#print('x_train shape:', x_train.shape)
#print('x_test shape:', x_test.shape)

print('Building model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Model complete...')

In [ ]:
max_words = 1000
batch_size = 32
epochs = 5

print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

print('Building model...')
model = Sequential()
model.add(Embedding(1000, 100))
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#model.add(Dense(512, input_shape=(max_words,)))
#model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Conv1D(filters=100,kernel_size=3, strides=1, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=2,
                    validation_split=0.1)
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [13]:
max_features = 10000
maxlen = 400
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

#print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
#x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
#x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
25000 train sequences
25000 test sequences
x_train shape: (25000L, 400L)
x_test shape: (25000L, 400L)


In [14]:
x_train = x_train[:2500]
x_test = x_test[:2500]

In [30]:
from __future__ import print_function
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.datasets import imdb


max_features = 20000
# cut texts after this number of words
# (among top max_features most common words)
maxlen = 400
batch_size = 32

print('Loading data...')
#(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
#print(len(x_train), 'train sequences')
#print(len(x_test), 'test sequences')

print("Pad sequences (samples x time)")
#x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
#x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
#y_train = np.array(y_train)[:2500]
#y_test = np.array(y_test)[:2500]

model = Sequential()
model.add(Embedding(max_features, 100, input_length=maxlen))
model.add(Bidirectional(LSTM(50)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
#model.fit(x_train, y_train,
model.fit(xx, y_train,
          batch_size=batch_size,
          epochs=20,
          verbose=2,
          validation_data=[xx_test, y_test])

Loading data...
Pad sequences (samples x time)
x_train shape: (504, 4149)
x_test shape: (127, 4149)
Train...
Train on 504 samples, validate on 127 samples
Epoch 1/20
12s - loss: -1.2310e+00 - acc: 0.1508 - val_loss: -5.9484e+00 - val_acc: 0.1732
Epoch 2/20
11s - loss: -1.6123e+01 - acc: 0.1548 - val_loss: -2.1132e+01 - val_acc: 0.1732
Epoch 3/20
11s - loss: -3.0564e+01 - acc: 0.1548 - val_loss: -3.4019e+01 - val_acc: 0.1732
Epoch 4/20
11s - loss: -3.8110e+01 - acc: 0.1548 - val_loss: -3.4019e+01 - val_acc: 0.1732
Epoch 5/20
12s - loss: -3.8193e+01 - acc: 0.1548 - val_loss: -3.4019e+01 - val_acc: 0.1732
Epoch 6/20
11s - loss: -3.8206e+01 - acc: 0.1548 - val_loss: -3.4019e+01 - val_acc: 0.1732
Epoch 7/20
12s - loss: -3.8211e+01 - acc: 0.1548 - val_loss: -3.4019e+01 - val_acc: 0.1732
Epoch 8/20
11s - loss: -3.8211e+01 - acc: 0.1548 - val_loss: -3.4019e+01 - val_acc: 0.1732
Epoch 9/20
15s - loss: -3.8211e+01 - acc: 0.1548 - val_loss: -3.4019e+01 - val_acc: 0.1732
Epoch 10/20
15s - loss: -3

Exception KeyboardInterrupt in 'zmq.backend.cython.message.Frame.__dealloc__' ignored
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\pedro.castanha\AppData\Local\Continuum\Anaconda2\lib\site-packages\IPython\core\ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Users\pedro.castanha\AppData\Local\Continuum\Anaconda2\lib\site-packages\IPython\core\ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "C:\Users\pedro.castanha\AppData\Local\Continuum\Anaconda2\lib\site-packages\IPython\core\ultratb.py", line 376, in _fixed_getinnerframes
    lines = ulinecache.getlines(file)[start:end]
  File "C:\Users\pedro.castanha\AppData\Local\Continuum\Anaconda2\lib\site-packages\IPython\utils\ulinecache.py", line 37, in getlines
    return [l.decode(encoding, 'replace') for l in lines]
  File "C:\Users\pedro.castanha\AppData\Local\Continuum\Anaconda2\lib\encodings\utf_8.py", line 15, in decode
    def decode(input, errors='strict'):
KeyboardInterrupt


IndexError: string index out of range

In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 400, 100)          2000000   
_________________________________________________________________
bidirectional_7 (Bidirection (None, 100)               60400     
_________________________________________________________________
dropout_7 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 2,060,501.0
Trainable params: 2,060,501.0
Non-trainable params: 0.0
_________________________________________________________________
